## Automated anisotropic resistivity inversion for efficient formation evaluation and uncertainty quantification

### Misael M. Morales, Oriyomi Raheem, Ali Eghbali, Michael Pyrcz, Carlos Torres-Verdin
***

In [7]:
from main import *
ari = ARI()
case1, case2 = ari.load_data()


------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.2.2+cu121 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080
------------------------------------------------------------



### Case 1

In [ ]:
case1_qinv = ari.quadratic_inversion(case1)

In [ ]:
inv1 = ari.resistivity_inversion(case1, x0=[0.5,1.45], Rvsh=2.813, Rhsh=0.775, 
                                 Wd_matrix=True, tolerance=1e-10, maxiter=1e5, 
                                 lambda_reg=1e-8, method='CG', bounds=None)

ari.plot_inversion_results(inv1, figsize=(20,12))

In [ ]:
inv1 = ari.resistivity_inversion(case1, x0=[0.5, 1.45], Rvsh=2.813, Rhsh=0.775,
                                 Wd_matrix=True, tolerance=1e-10, maxiter=1e5,
                                 lambda_reg=1e-8, method='L-BFGS-B', bounds=[(0,1),(None,None)])

ari.plot_inversion_results(inv1, figsize=(20,12))

### Case 2

In [ ]:
case2_qinv = ari.quadratic_inversion(case2)

In [ ]:
inv2 = ari.resistivity_inversion(case2, x0=[0.5,1.45], Rvsh=4.043, Rhsh=0.674,
                                 Wd_matrix=True, tolerance=1e-10, maxiter=1e5, 
                                 lambda_reg=1e-8, method='CG', bounds=None)

ari.plot_inversion_results(inv2, figsize=(20,12))

In [ ]:
inv2 = ari.resistivity_inversion(case2, x0=[0.5,1.45], Rvsh=4.043, Rhsh=0.674, 
                                 Wd_matrix=True, tolerance=1e-8, maxiter=1e5, 
                                 lambda_reg=1e-8, method='L-BFGS-B', bounds=[(0,1),(None,None)])

ari.plot_inversion_results(inv2, figsize=(20,12))

***
# Physics-Informed Artificial Neural Network
***

In [ ]:
from main import *
ari = ARI()
case1, case2 = ari.load_data()

In [ ]:
def plot_pinn_results(df, figsize=(15,10), subtitle:str=None):
    fig, axs = plt.subplots(1, 4, figsize=figsize, sharey=True)
    ax1, ax2, ax3, ax4 = axs

    ax11 = ax1.twiny()
    ari.plot_curve(ax1, df, 'GR', 0, 150, 'g', units='API', pad=8)
    ari.plot_curve(ax11, df, 'Csh_pred', 0, 1, 'k', units='v/v')

    ax21, ax22 = ax2.twiny(), ax2.twiny()
    ari.plot_curve(ax2, df, 'AT10', 0.2, 50, 'r', semilog=True, units='$\Omega\cdot m$', pad=8)
    ari.plot_curve(ax21, df, 'AT90', 0.2, 50, 'b', semilog=True, units='$\Omega\cdot m$', pad=16)
    ari.plot_curve(ax22, df, 'Rss_pred', 0.2, 50, 'k', units='$\Omega\cdot m$', semilog=True)

    ax31, ax32 = ax3.twiny(), ax3.twiny()
    ari.plot_curve(ax31, df, 'Rv', 0.2, 100, 'r', semilog=True, units='$\Omega\cdot m$', pad=8)
    ari.plot_curve(ax32, df, 'Rv_sim', 0.2, 100, 'k', units='$\Omega\cdot m$', semilog=True)
    ari.plot_curve(ax3, df, 'Rv_err', 0, 100, 'darkred', alpha=0.5, units='%', pad=16)

    ax41, ax42 = ax4.twiny(), ax4.twiny()
    ari.plot_curve(ax41, df, 'Rh', 0.2, 100, 'b', semilog=True, units='$\Omega\cdot m$', pad=8)
    ari.plot_curve(ax42, df, 'Rh_sim', 0.2, 100, 'k', units='$\Omega\cdot m$', semilog=True)
    ari.plot_curve(ax4, df, 'Rh_err', 0, 100, 'darkblue', alpha=0.5, units='%', pad=16)

    [ax.grid(True, which='both') for ax in axs]
    ax1.invert_yaxis()
    if subtitle is None:
        title = 'PINN Inversion Results'
    else:
        title = 'PINN Inversion Results | {}'.format(subtitle)
    fig.suptitle(title, weight='bold')
    plt.tight_layout()
    plt.show()

In [ ]:
class LSTMari(nn.Module):
    def __init__(self):
        super(LSTMari, self).__init__()
        self.fc1 = nn.Linear(2, 64)
        self.rn1 = nn.LSTM(input_size=64,  hidden_size=128, num_layers=2, batch_first=True, dropout=0.1)
        self.rn2 = nn.LSTM(input_size=128, hidden_size=64,  num_layers=2, batch_first=True, dropout=0.1)
        self.fc3 = nn.Linear(64, 2)
        
    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))
        x, _ = self.rn1(x)
        x = nn.ReLU()(x)
        x, _ = self.rn2(x)
        x = nn.ReLU()(self.fc3(x))
        return x

In [ ]:
class siameseFNO(nn.Module):
    def __init__(self, Csh_constraint:bool=False):
        super(siameseFNO, self).__init__()
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(2,   64,  kernel_size=3, padding=1),
            nn.Conv1d(64,  256, kernel_size=3, padding=1),
            nn.Conv1d(256, 64,  kernel_size=3, padding=1),
        ])      
        self.output_layer = nn.Conv1d(64, 1, kernel_size=3, padding=1)
        self.Csh_constraint = Csh_constraint

    def complex_relu(self, x):
        return torch.complex(torch.relu(x.real), torch.relu(x.imag))
        
    def forward(self, x):
        x = x.to(torch.complex64).permute(1,0)
        x_freq = torch.fft.fft(torch.fft.fftshift(x), dim=-2)
        for layer in self.conv_layers:
            layer.weight = nn.Parameter(layer.weight.to(torch.complex64))
            layer.bias = nn.Parameter(layer.bias.to(torch.complex64))
            x_freq = self.complex_relu(layer(x_freq))
        x_out = torch.fft.ifft(torch.fft.ifftshift(x_freq), dim=-2)
        x_out = self.output_layer(torch.abs(x_out))
        x_out = x_out.permute(1,0)
        return x_out

In [ ]:
class PhysicsInformedLoss(torch.nn.Module):
    def __init__(self, lambda_reg=1e-8):
        super(PhysicsInformedLoss, self).__init__()
        self.lambda_reg = lambda_reg

    def forward(self, inputs, outputs):
        Rv_true = inputs[:, 0]
        Rh_true = inputs[:, 1]

        Rvsh = torch.quantile(Rv_true, q=0.9)
        Rhsh = torch.quantile(Rh_true, q=0.9)

        Csh = outputs[:, 0]
        Rss = outputs[:, 1]
    
        Rvsim = (Csh*Rvsh + (1-Csh)*Rss)
        Rhsim = 1 / (Csh/Rhsh + (1-Csh)/Rss)

        Rverr = torch.abs((Rv_true - Rvsim)/Rv_true)
        Rherr = torch.abs((Rh_true - Rhsim)/Rh_true)
        costf = torch.cat((Rverr, Rherr), dim=0)
        loss  = torch.norm(costf, p=2) + self.lambda_reg*torch.norm(inputs, p=2)
        return loss

In [ ]:
d = lasio.read('well1.las').df()[['AT10','AT30','AT60','AT90','GR','RV72H_1D_FLT','RH72H_1D_FLT']].dropna()
#d = lasio.read('well2.LAS').df()[['AT10','AT30','AT60','AT90','HCGR','RV72_1DF','RH72_1DF']].dropna()

column_names = ['AT10', 'AT30', 'AT60', 'AT90', 'GR', 'Rv', 'Rh']
data = d.rename(columns=dict(zip(d.columns, column_names)))

res_aniso  = data[['Rv','Rh']][data.index < 10200]

Rvsh_win = np.nan_to_num(np.array(pd.Series(data['Rv']).rolling(250).max().tolist()), nan=data['Rv'].mean())
Rhsh_win = np.nan_to_num(np.array(pd.Series(data['Rh']).rolling(250).max().tolist()), nan=data['Rh'].mean())
Csh_lin  = (data['GR'] - data['GR'].min())/(data['GR'].max() - data['GR'].min())

R_min = np.min([data['Rh'].min(), data['Rv'].min()])
R_max = np.max([data['Rh'].max(), data['Rv'].max()])

In [ ]:
inputs         = torch.tensor(res_aniso.values, dtype=torch.float32)
dataset        = TensorDataset(inputs)
train_percent  = 0.8
xtrain, xvalid = random_split(dataset, [int(train_percent*len(dataset)), len(dataset)-int(train_percent*len(dataset))])
trainloader    = DataLoader(xtrain, batch_size=100, shuffle=False)
validloader    = DataLoader(xvalid, batch_size=100, shuffle=True)

csh_model  = siameseFNO(Csh_constraint=True)
rss_model  = siameseFNO()
criterion  = PhysicsInformedLoss()
optimizer  = torch.optim.AdamW([{'params': csh_model.parameters()}, {'params': rss_model.parameters()}], 
                               lr=1e-3, weight_decay=1e-6)

epochs = 251
train_loss, valid_loss = [], []

for epoch in range(epochs):
    
    epoch_train_loss = []
    csh_model.train()
    rss_model.train()
    for batch in trainloader:
        optimizer.zero_grad()
        x = batch[0]
        ycsh = csh_model(x)
        yrss = rss_model(x)
        y = torch.cat((ycsh, yrss), dim=1)
        loss = criterion(x, y)
        loss.backward()
        optimizer.step()
        epoch_train_loss.append(loss.item())
    train_loss.append(np.mean(epoch_train_loss))

    csh_model.eval()
    rss_model.eval()
    epoch_valid_loss = []
    with torch.no_grad():
        x = next(iter(validloader))[0]
        ycsh = csh_model(x)
        yrss = rss_model(x)
        y = torch.cat((ycsh, yrss), dim=1)
        loss = criterion(x, y)
        epoch_valid_loss.append(loss.item())
    valid_loss.append(np.mean(epoch_valid_loss))

    if epoch % 50 == 0:
        print('Epoch: {} | Loss: {:.4f} | Valid Loss: {:.4f}'.format(epoch, train_loss[-1], valid_loss[-1]))

In [ ]:
plt.figure(figsize=(6,3))
plt.plot(range(epochs), train_loss, label='Trianing', c='tab:blue')
plt.plot(range(epochs), valid_loss, label='Validation', c='tab:orange')
plt.legend(facecolor='lightgrey', edgecolor='k')
plt.grid(True, which='both')
plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.xlim(-2, epochs+2)
plt.tight_layout()
plt.show()

In [ ]:
csh_pred = csh_model(inputs).detach().numpy().squeeze()
rss_pred = rss_model(inputs).detach().numpy().squeeze()
print('Csh: min={:.3f} | max={:.3f}'.format(csh_pred.min(), csh_pred.max()))

Rvsh_win, Rhsh_win = np.quantile(res_aniso['Rv'], q=0.9), np.quantile(res_aniso['Rh'], q=0.9)

Rv_sim = (csh_pred*Rvsh_win + (1-csh_pred)*rss_pred)
Rh_sim = 1 / (csh_pred/Rhsh_win + (1-csh_pred)/rss_pred)

Rv_err = np.abs((Rv_sim - res_aniso['Rv'].values)/res_aniso['Rv'].values) * 100
Rh_err = np.abs((Rh_sim - res_aniso['Rh'].values)/res_aniso['Rh'].values) * 100

sol = pd.DataFrame({'Csh_pred':csh_pred, 'Rss_pred':rss_pred,
                    'Rv_sim':Rv_sim, 'Rh_sim':Rh_sim,
                    'Rv_err':Rv_err, 'Rh_err':Rh_err}, index=res_aniso.index)

results = pd.concat([data, sol], axis=1)
plot_pinn_results(results, subtitle='Chevron 2009')

***
# END
***

In [ ]:
from main import *
ari = ARI()
case1, case2 = ari.load_data()

In [ ]:
ali_cg = loadmat('Ali/sol.mat', simplify_cells=True)['sol']
ali_Csh = ali_cg[0,:]
ali_Rss = ali_cg[1,:]

ali_Rvsh = 2.78
ali_Rhsh = 0.60

Rv_sim_ali = ali_Csh*ali_Rvsh + (1-ali_Csh)*ali_Rss
Rh_sim_ali = 1/(ali_Csh/ali_Rhsh + (1-ali_Csh)/ali_Rss)

ali = pd.read_csv('Ali/RVRH.csv', header=None)
d = ali.iloc[1:2156, 0].values
c = ali.iloc[1:2156, 14].values
r = ali.iloc[1:2156, 15].values
ali_sol = pd.DataFrame({'Csh_ALI':c, 'Rss_ALI':r}, index=d)
ali_sol['Rv_sim_ALI'] = c*ali_Rvsh + (1-c)*r
ali_sol['Rh_sim_ALI'] = 1/(c/ali_Rhsh + (1-c)/r)

In [ ]:
def quadratic_inversion(df, Rvsh=None, Rhsh=None):
    quad_inv = []
    if Rvsh is None:
        Rvsh = df['Rv'].iloc[np.argmax(df['GR'])]
    if Rhsh is None:
        Rhsh = df['Rh'].iloc[np.argmax(df['GR'])]
    for _, row in df.iterrows():
        Rv, Rh = row['Rv'], row['Rh']
        a = Rh*Rvsh - Rh*Rhsh
        b = Rv**2 + Rvsh*Rhsh - 2*Rh*Rhsh
        c = Rv*Rhsh - Rh*Rhsh
        qsol = np.roots([a,b,c])
        if len(qsol) == 1:
            quad_inv.append({'Rss_q':qsol[0], 'Csh_q':np.nan})
        elif len(qsol) == 2:
            quad_inv.append({'Rss_q':qsol[0], 'Csh_q':qsol[1]})
        else:
            quad_inv.append({'Rss_q':np.nan, 'Csh_q':np.nan})
    quad_inv = pd.DataFrame(quad_inv, index=df.index)
    return quad_inv

In [ ]:
d = lasio.read('well1.las').df()[['AT10','AT30','AT60','AT90','GR','RV72H_1D_FLT','RH72H_1D_FLT']].dropna()
column_names = ['AT10', 'AT30', 'AT60', 'AT90', 'GR', 'Rv', 'Rh']
data = d.rename(columns=dict(zip(d.columns, column_names)))
max_depth = 10188.75

R_min = np.min([data['Rh'].min(), data['Rv'].min()])
R_max = np.max([data['Rh'].max(), data['Rv'].max()])
print('R_min={:.2f} | R_max={:.2f}'.format(R_min, R_max))

mask1 = np.logical_and(d.index >= 9650, d.index <= 9822)
mask2 = np.logical_and(d.index >= 9822, d.index <= 10085)
mask3 = np.logical_and(d.index >= 10085, d.index <= max_depth)
masks = [mask1, mask2, mask3]

Rvsh_win = np.zeros(len(data))
Rhsh_win = np.zeros(len(data))

for i, mask in enumerate(masks):
    Rvsh_win[mask] = np.quantile(data['Rv'][mask], 0.8)
    Rhsh_win[mask] = np.quantile(data['Rh'][mask], 0.8)

data['Rvsh_win'], data['Rhsh_win'] = Rvsh_win, Rhsh_win
data['Csh_lin']  = (data['GR'] - data['GR'].min())/(data['GR'].max() - data['GR'].min())

qinv = np.abs(quadratic_inversion(data))
data = data.join(qinv)[data.index<=max_depth]
data

In [ ]:
def plot_optim_sol(xx,yy,zz,xall,objective,const,constraints):
        x0, xf = xall[0], xall[-1]
        fig = plt.figure(figsize=(12,5))

        ax1 = fig.add_subplot(121)
        im = ax1.contourf(xx, yy, zz, levels=20, cmap='jet')
        ax1.contour(xx, yy, zz, levels=50, colors='k', linewidths=0.5)
        ax1.scatter(x0[0], x0[1], c='r', marker='o', s=50, label='Initial')
        ax1.plot(xall[:,0], xall[:,1], ls='-', color='k', marker='.', markerfacecolor='w', markersize=10, label='history')
        ax1.scatter(xf[0], xf[1], c='r', marker='*', s=100, label='Optimal')
        ax1.vlines([constraints[0][0], constraints[0][1]], constraints[1][0], constraints[1][1], colors='k', ls='--', label='c1')
        ax1.hlines([constraints[1][0], constraints[1][1]], constraints[0][0], constraints[0][1], colors='k', ls=':', label='c2')
        ax1.legend(facecolor='lightgrey', edgecolor='k', loc='lower center', ncols=5, bbox_to_anchor=(0.5, -0.25))
        ax1.set(yscale='log', xlabel='$C_{sh}$', ylabel='$R_{ss}$', title='Objective Function | $x^*=({:.3f},{:.3f})$'.format(xf[0], xf[1]))
        cb=plt.colorbar(im, pad=0.04, fraction=0.046); cb.set_label('Objective Value', rotation=270, labelpad=15)
        
        ax2 = fig.add_subplot(122, projection='3d')
        ax2.plot_surface(xx, yy, zz, cmap='jet', alpha=0.8)
        ax2.contour(xx, yy, zz, levels=10, colors='k', linewidths=0.5)
        ax2.scatter(x0[0], x0[1], objective(x0, *const), c='r', marker='o', s=50)
        ax2.plot(xall[:,0], xall[:,1], objective(xall.T, *const), ls='-', color='k', marker='.', markerfacecolor='w', markersize=10)
        ax2.scatter(xf[0], xf[1], objective(xf, *const), c='r', marker='*', s=100)
        ax2.set(xlabel='$C_{sh}$', ylabel='$R_{ss}$', zlabel='Objective Value', title='Objective Function | $x^*=({:.3f},{:.3f})$'.format(xf[0], xf[1]))
        plt.tight_layout()
        plt.show()
        return None

In [ ]:
res_aniso = data[['Rv', 'Rh', 'Csh_lin', 'Csh_q', 'Rss_q', 'Rvsh_win', 'Rhsh_win']].join(ali_sol.iloc[:,:2])

lambda_reg = 0
def objective(variables, *args):
    def reg(x, l, o):
        return l*np.linalg.norm(x, ord=o)
    Csh, Rs = variables
    Rv, Rh, Rvsh, Rhsh = args[0], args[1], args[2], args[3]
    eq1 = (Csh*Rvsh + (1-Csh)*Rs) - Rv
    eq2 = (Csh/Rhsh + (1-Csh)/Rs) - (1/Rh)   
    eqs = np.array([eq1/Rv, eq2/Rh])
    return np.linalg.norm(eqs, ord=2) + reg(variables,0,2)

def jacobian(variables, *args):
    return Jacobian(lambda x: objective(x, *args))(variables).ravel()

def hessian(variables, *args):
    return Hessian(lambda x: objective(x, *args))(variables)

bounds = [(0,1), (1e-3, 5e3)]
cCsh = optimize.NonlinearConstraint(lambda x: x[0], lb=bounds[0][0], ub=bounds[0][1])
cRss = optimize.NonlinearConstraint(lambda x: x[1], lb=bounds[1][0], ub=bounds[1][1])

xhist = []
def callback(x, *args):
    xhist.append(x)

row = 812
x0 = tuple(res_aniso[['Csh_q', 'Rss_q']].iloc[row].values)
constants = tuple(res_aniso[['Rv', 'Rh', 'Rvsh_win', 'Rhsh_win']].iloc[row].values)

sol = []
res = optimize.minimize(objective,
                        x0          = x0,
                        args        = constants,
                        method      = 'trust-constr',
                        jac         = jacobian,
                        hess        = hessian,
                        constraints = (cCsh,cRss),
                        tol         = 1e-6,
                        callback    = callback,
                        options     = {'maxiter':1e3, 'disp':True})
sol.append({'Rv':constants[0], 'Rh':constants[1], 'Csh_pred':res.x[0], 'Rss_pred':res.x[1],
            'fun':res.fun, 'nfev':res.nfev, 'jac_norm':linalg.norm(res.jac),
            'Rv_sim':res.x[0]*constants[2] + (1-res.x[0])*res.x[1],
            'Rh_sim':1/(res.x[0]/constants[3] + (1-res.x[0])/res.x[1])})

xall = np.array([x0]+xhist)
xx, yy = np.meshgrid(np.linspace(-0.25, 1.25, 100), np.linspace(5e-4, 1e4, 100))
zz = np.array([objective([x,y], *constants) for x,y in zip(xx.ravel(), yy.ravel())]).reshape(xx.shape)

plot_optim_sol(xx,yy,zz,xall,objective,constants,bounds)

In [ ]:
sol

In [ ]:
res_aniso = data[['Rv', 'Rh', 'Csh_lin', 'Csh_q', 'Rss_q', 'Rvsh_win', 'Rhsh_win']].join(ali_sol.iloc[:,:2])

lambda_reg = 1e-10
sol = []

def objective(variables, *args):
    def reg(x, l, o):
        return l*np.linalg.norm(x, ord=o)
    def tsvd(x, o):
        d = np.array(x).reshape(-1,1)
        U, S_, Vt = linalg.svd(d, full_matrices=False)
        S = np.diag(S_)
        return Vt.T @ linalg.inv(S) @ U.T[:o,:o]
    Csh, Rs = variables   
    Rv, Rh, Rvsh, Rhsh = args[0], args[1], args[2], args[3]
    eq1 = (Csh*Rvsh + (1-Csh)*Rs) - Rv
    eq2 = (Csh/Rhsh + (1-Csh)/Rs) - (1/Rh)   
    eqs = [eq1/Rv, eq2/Rh]
    return linalg.norm(eqs,2) + reg(variables,1e-3,2)

def jacobian(variables, *args):
    return Jacobian(lambda x: objective(x, *args))(variables).ravel()

def hessian(variables, *args):
    return Hessian(lambda x: objective(x, *args))(variables)

bounds = [(0,1), (1e-3, 5e3)]
cCsh = optimize.NonlinearConstraint(lambda x: x[0], lb=bounds[0][0], ub=bounds[0][1])
cRss = optimize.NonlinearConstraint(lambda x: x[1], lb=bounds[1][0], ub=bounds[1][1])

def constraint(variables):
    Csh, Rss = variables
    Csh = np.clip(Csh, 0, 1)
    return Csh, Rss

for _, row in res_aniso.iterrows():   
    x0        = (row['Csh_q'], row['Rss_q'])
    constants = (row['Rv'], row['Rh'], row['Rvsh_win'], row['Rhsh_win'])
    res = optimize.minimize(objective,
                            x0          = x0,
                            args        = constants,
                            method      = 'trust-constr',
                            jac         = jacobian,
                            hess        = hessian,
                            constraints = (cCsh,cRss),
                            #bounds      = [(0.0001,0.9999),(2e-2, 2e3)], #(R_min, R_max)
                            tol         = 1e-6,
                            options     = {'maxiter':1e3})
    sol.append({'Rv':constants[0], 'Rh':constants[1], 'Csh_pred':res.x[0], 'Rss_pred':res.x[1],
                'fun':res.fun, 'nfev':res.nfev, 'jac':linalg.norm(res.jac)})
sol = pd.DataFrame(sol, index=res_aniso.index)

sol['Rv_sim'] = sol['Csh_pred']*data['Rvsh_win'] + (1-sol['Csh_pred'])*sol['Rss_pred']
sol['Rh_sim'] = 1/(sol['Csh_pred']/data['Rhsh_win'] + (1-sol['Csh_pred'])/sol['Rss_pred'])
sol.head()

In [ ]:
fig, axs = plt.subplots(1,5, figsize=(16.5,10), sharey=True, width_ratios=[0.6,0.6,0.6,1,1])
ax1, ax2, ax3, ax4, ax5 = axs
colors = ['darkviolet','royalblue','firebrick']

ax1.plot(data['GR'], data.index, c='g', label='GR')
ax1.grid(True, which='both')
ax1.set(xlim=(20,120), title='GR')
ax1.hlines(10188.75, 0, case1['GR'].max(), color='k', lw=5)

ax2.plot(data['Rv'], data.index, c='k', label='Rv')
ax2.plot(Rv_sim_ali, ali.iloc[1:2156,0], c='b', ls='--', label='Ali_sim')
ax2.plot(sol['Rv_sim'], sol.index, c='r', ls='--', label='Simulated')
ax2.set(xscale='log', title='Rv')
ax2.grid(True, which='both')
ax2.legend(loc='upper left', facecolor='lightgrey', edgecolor='k')
ax2.hlines(10188.75, 0, sol['Rss_pred'].max(), color='k', lw=5)

ax3.plot(data['Rh'], data.index, c='k', label='Rh')
ax3.plot(Rh_sim_ali, ali.iloc[1:2156,0], c='b', ls='--', label='Ali_sim')
ax3.plot(sol['Rh_sim'], sol.index, c='r', ls='--', label='Simulated')
ax3.set(xscale='log', title='Rh')
ax3.grid(True, which='both')
ax3.legend(loc='upper left', facecolor='lightgrey', edgecolor='k')
ax3.hlines(10188.75, 0, sol['Rss_pred'].max(), color='k', lw=5)

ax4.plot(ali_Csh, ali.iloc[1:2156,0], label='Ali_1', c='b')
ax4.plot(ali.iloc[:,14], ali.iloc[:,0], label='Ali_2')
ax4.plot(sol['Csh_pred'], sol.index, label='Mine', ls='--', c='r')
ax4.set(title='Csh', xlim=(0,1))
ax4.legend(loc='upper left', facecolor='lightgrey', edgecolor='k')
ax4.grid(True, which='both')
ax4.hlines(10188.75, 0, 1, color='k', lw=5)

ax5.plot(ali_Rss, ali.iloc[1:2156,0], label='Ali_1', c='b')
ax5.plot(ali.iloc[:,15], ali.iloc[:,0], label='Ali_2')
ax5.plot(sol['Rss_pred'], sol.index, label='Mine', ls='--', c='r')
ax5.set(xscale='log', title='Rss'); 
ax5.legend(loc='upper left', facecolor='lightgrey', edgecolor='k')
ax5.grid(True, which='both')
ax5.hlines(10188.75, 0, sol['Rss_pred'].max(), color='k', lw=5)

ax1.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 6, figsize=(15,10))
ax1, ax2, ax3, ax4, ax5, ax6 = axs

ax1.plot(sol['Rv'], sol.index, c='k', lw=2.75, label='Rv_true')
ax1.plot(sol['Rv_sim'], sol.index, c='r', ls='--', label='Rv_sim')
ax1.set(title='Rv_sim'); ax1.grid(True, which='both'); ax1.set_xscale('log')
ax1.legend(facecolor='lightgrey', edgecolor='k', loc='upper right')

ax2.plot(sol['Rh'], sol.index, c='k', lw=2.75, label='Rh_true')
ax2.plot(sol['Rh_sim'], sol.index, c='r', ls='--', label='Rh_sim')
ax2.set(title='Rh_sim'); ax2.grid(True, which='both'); ax2.set_xscale('log')
ax2.legend(facecolor='lightgrey', edgecolor='k', loc='upper right')

ax3.plot(sol['Rv'], sol.index, c='k', lw=2.75, label='Rv_true')
ax3.plot(Rv_sim_ali, ali.iloc[:2155,0], c='b', ls='--', label='ALI_Rv_sim')
ax3.set(title='Rv_sim'); ax3.grid(True, which='both'); ax3.set_xscale('log')
ax3.legend(facecolor='lightgrey', edgecolor='k', loc='upper right')

ax4.plot(sol['Rh'], sol.index, c='k', lw=2.75, label='Rh_true')
ax4.plot(Rh_sim_ali, ali.iloc[:2155,0], c='b', ls='--', label='ALI_Rh_sim')
ax4.set(title='Rh_sim'); ax4.grid(True, which='both'); ax4.set_xscale('log')
ax4.legend(facecolor='lightgrey', edgecolor='k', loc='upper right')

ax5.plot(sol['Rv'], sol.index, c='k', lw=2.75, label='Rv_true')
ax5.plot(Rv_sim_ali, ali.iloc[:2155,0], c='b', ls='--', label='ALI_Rv_sim')
ax5.plot(sol['Rv_sim'], sol.index, c='r', ls='--', label='Rv_sim')
ax5.set(title='Rv_sim'); ax5.grid(True, which='both'); ax5.set_xscale('log')
ax5.legend(facecolor='lightgrey', edgecolor='k', loc='upper right')

ax6.plot(sol['Rh'], sol.index, c='k', lw=2.75, label='Rh_true')
ax6.plot(Rh_sim_ali, ali.iloc[:2155,0], c='b', ls='--', label='ALI_Rh_sim')
ax6.plot(sol['Rh_sim'], sol.index, c='r', ls='--', label='Rh_sim')
ax6.set(title='Rh_sim'); ax6.grid(True, which='both'); ax6.set_xscale('log')
ax6.legend(facecolor='lightgrey', edgecolor='k', loc='upper right')

for ax in axs:
    ax.hlines(10188.75, 0, sol['Rss_pred'].max(), color='k', lw=3.5)
    ax.invert_yaxis()
    
plt.tight_layout()
plt.show()

In [ ]:
df = lasio.read('well1.las').df()

fig, axs = plt.subplots(1, 10, figsize=(17.5,10), sharey=True)
ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9 = axs

ax01 = ax0.twiny()
ari.plot_curve(ax0, df, 'CALI', 12, 24, 'dodgerblue', units='in', fill=True, semilog=True, pad=0)
ari.plot_curve(ax01, df, 'GR', 10, 150, 'g', units='API', pad=8)

ax11, ax12 = ax1.twiny(), ax1.twiny()
ari.plot_curve(ax1, data, 'Csh_lin', 0, 1, 'k', units='v/v', pad=0)
ari.plot_curve(ax11, data, 'Csh_q', 0, 1, 'gray', units='v/v', pad=8)
ari.plot_curve(ax12, sol, 'Csh_pred', 0, 1, 'r', units='v/v', pad=16)

ax21, ax22 = ax2.twiny(), ax2.twiny()
ari.plot_curve(ax2, df, 'TNPH', 1, 0, 'b', units='PU', pad=0)
ari.plot_curve(ax21, df, 'RHOZ', 1.65, 2.65, 'maroon', units='g/cc', pad=8)
ari.plot_curve(ax22, df, 'PE', -5, 5, 'm', units='b/e', pad=16)

ax31, ax32 = ax3.twiny(), ax3.twiny()
ari.plot_curve(ax3, df, 'AT10', 0.2, 100, 'darkred', semilog=True, units='$\Omega\cdot m$', pad=0)
ari.plot_curve(ax31, df, 'AT90', 0.2, 100, 'darkblue', semilog=True, units='$\Omega\cdot m$', pad=8)
ari.plot_curve(ax32, sol, 'Rss_pred', 0.2, 100, 'r', semilog=True, units='$\Omega\cdot m$', pad=16)

ax41 = ax4.twiny()
ari.plot_curve(ax4, ali_sol, 'Csh_ALI', 0, 1, 'b', units='v/v', pad=0)
ari.plot_curve(ax41, sol, 'Csh_pred', 0, 1, 'r', ls='--', units='v/v', pad=8)

ax51 = ax5.twiny()
ari.plot_curve(ax5, ali_sol, 'Rss_ALI', 0.2, 100, 'b', semilog=True, units='$\Omega\cdot m$', pad=0)
ari.plot_curve(ax51, sol, 'Rss_pred', 0.2, 100, 'r', ls='--', semilog=True, units='$\Omega\cdot m$', pad=8)

ax61, ax62 = ax6.twiny(), ax6.twiny()
ari.plot_curve(ax6, df, 'RV72H_1D_FLT', 0.2, 100, 'k', s=4, semilog=True, units='$\Omega\cdot m$', pad=0)
ari.plot_curve(ax61, ali_sol, 'Rv_sim_ALI', 0.2, 100, 'b', semilog=True, units='$\Omega\cdot m$', pad=8)
ari.plot_curve(ax62, sol, 'Rv_sim', 0.2, 100, 'r', ls='--', semilog=True, units='$\Omega\cdot m$', pad=16)

ax71, ax72 = ax7.twiny(), ax7.twiny()
ari.plot_curve(ax7, df, 'RH72H_1D_FLT', 0.2, 100, 'k', s=4, semilog=True, units='$\Omega\cdot m$', pad=0)
ari.plot_curve(ax71, ali_sol, 'Rh_sim_ALI', 0.2, 100, 'b', semilog=True, units='$\Omega\cdot m$', pad=8)
ari.plot_curve(ax72, sol, 'Rh_sim', 0.2, 100, 'r', ls='--', semilog=True, units='$\Omega\cdot m$', pad=16)

ax81, ax82, ax83 = ax8.twiny(), ax8.twiny(), ax8.twiny()
ari.plot_curve(ax8, data, 'Rv', 0.2, 100, 'k', s=4, semilog=True, units='$\Omega\cdot m$', pad=0)
ari.plot_curve(ax81, sol, 'Rv_sim', 0.2, 100, 'darkred', ls='--', semilog=True, units='$\Omega\cdot m$', pad=8)
ari.plot_curve(ax82, data, 'Rh', 0.2, 100, 'k', s=4, semilog=True, units='$\Omega\cdot m$', pad=16)
ari.plot_curve(ax83, sol, 'Rh_sim', 0.2, 100, 'darkblue', ls='--', semilog=True, units='$\Omega\cdot m$', pad=24)

ax91, ax92 = ax9.twiny(), ax9.twiny()
ari.plot_curve(ax9, sol, 'fun', 1e-6, 1e0, 'k', s=1, semilog=True, units='/', pad=0)
ari.plot_curve(ax91, sol, 'jac_norm', 0, 2.5, 'darkmagenta', s=1, units='/', pad=8, alpha=0.5)
ari.plot_curve(ax92, sol, 'nfev', 10, 150, 'darkgreen', s=1, units='/', pad=16, alpha=0.5)

ax1.set_ylim(10190, 9650)

plt.tight_layout()
plt.show()